In [5]:
import h5py
import pandas as pd
import numpy as np
import os

In [4]:
!pwd

/home/jovyan/workspace/Desktop/mite_outputs


In [6]:
def populate_tables(data_folder_name='test_temp_bo/dades', 
                   tracers='lag'):
    #########################################
    """
    This function create the txt files with the strain rate data
    
    Arguments:
    - data_folder_name (str): folder where the raw h5 files are stored. 
        Default is 'outputs-fixed-values'.
    - tracers (str): name of tracers. Default is 'lag'.
    
    Returns:
    - Folder called txt_outputs with all the files.
    """
    #########################################
    
    # Check if the data folder exists
    if not os.path.exists(data_folder_name):
        raise FileNotFoundError(f"Folder '{data_folder_name}' not found.")
    
    # count number of times, checking the number that files that starts with prefix
    prefix = f'{tracers}_velocitygradient'
    files = os.listdir(data_folder_name)
    num_files = sum([1 for file in files if file.startswith(prefix)])
    print(num_files)
    # loop by number of files
    time = 0
    while time < num_files:
        print(f'time: {time}')
        ## COORDINATES ##
        # open Coordinates H5 
        h5_file_coord =  f'./{data_folder_name}/{tracers}-{time}.h5'
        if not os.path.exists(h5_file_coord):
                print(f"Warning: H5 file not found at '{h5_file_coord}'. Skipping this time step.")
                time += 1
                continue
        
        with h5py.File(h5_file_coord, 'r') as file:
            # create a variable name to store coord
            dataset_name = f'np_array_{time}'

            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                globals()[dataset_name] = file[key][:] 
                
        # create a table for each time and populate them with numpy array 
        # first create the name of the table
        df_name_by_t = f'df_t_{time}'
        # create the table
        globals()[df_name_by_t] = pd.DataFrame(globals()[dataset_name], columns=['x', 'y', 'z'])
        #add index in first column
        globals()[df_name_by_t].insert(0, 'index', globals()[df_name_by_t].index)
        #print(df_name_by_t)

        ## Velocity Gradient TENSOR ##
        # open the Strain Rate Tensor H5 files 
        h5_file_path =  f'./{data_folder_name}/{prefix}-{time}.h5'
        if not os.path.exists(h5_file_path):
                print(f"Warning: H5 Strain Rate Tensor file not found at '{h5_file_path}'. Skipping this time step.")
                time += 1
                continue
        
        with h5py.File(h5_file_path, 'r') as file:
            # create a variable name for storing strain rate tensor array
            dataset_name = f'np_array_{time}'

            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                globals()[dataset_name] = file[key][:] 
            
        # [a00,a01,a02,a10,a11,a12,a20,a21,a22]
        #globals()[dataset_name][0]
        globals()[df_name_by_t]['VG00'] = globals()[dataset_name][:, 0]
        globals()[df_name_by_t]['VG01'] = globals()[dataset_name][:, 1]
        globals()[df_name_by_t]['VG02'] = globals()[dataset_name][:, 2]
        globals()[df_name_by_t]['VG10'] = globals()[dataset_name][:, 3]
        globals()[df_name_by_t]['VG11'] = globals()[dataset_name][:, 4]
        globals()[df_name_by_t]['VG12'] = globals()[dataset_name][:, 5]
        globals()[df_name_by_t]['VG20'] = globals()[dataset_name][:, 6]
        globals()[df_name_by_t]['VG21'] = globals()[dataset_name][:, 7]
        globals()[df_name_by_t]['VG22'] = globals()[dataset_name][:, 8]
          
        ## save the df as txt
        output_folder = 'txt_outputs_test'
        output_file = os.path.join(output_folder, f'results{df_name_by_t}.txt')
            
        # check if the folder exists; if not, create it
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print('out folder: ', output_folder)
            
        # save to tab-separated values (.txt) file
        globals()[df_name_by_t].to_csv(output_file, sep='\t', index=False)

        #########
            
            
        print(f'hi {df_name_by_t}') 
        print(globals()[df_name_by_t].tail())
        time += 1
        
            
        

In [7]:
populate_tables()

97
time: 0
hi df_t_0
     index       x       y      z  VG00  VG01  VG02  VG10  VG11  VG12  VG20  \
995    995  2000.0  2750.0 -400.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
996    996  2000.0  2750.0 -450.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
997    997  2000.0  2750.0 -500.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
998    998  2000.0  2750.0 -550.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
999    999  2000.0  2750.0 -600.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

     VG21  VG22  
995   0.0   0.0  
996   0.0   0.0  
997   0.0   0.0  
998   0.0   0.0  
999   0.0   0.0  
time: 1
hi df_t_1
     index            x            y           z          VG00          VG01  \
995    995  1974.690702  2753.099009 -398.284874  2.742750e-16 -8.016316e-16   
996    996  1971.581496  2754.701738 -448.358760  2.508237e-16 -9.663784e-16   
997    997  1968.185143  2756.240981 -498.518940  3.368112e-16 -6.362836e-16   
998    998  1964.919896  2757.703396 -548.833839  2.977921

hi df_t_14
     index            x            y           z          VG00          VG01  \
995    995  1913.551571  2760.450798 -387.804222 -9.647261e-17 -3.141183e-16   
996    996  1899.741541  2763.617449 -439.402483 -1.716794e-16 -3.475075e-16   
997    997  1886.446399  2766.859336 -491.333735 -2.126004e-16 -2.691854e-16   
998    998  1874.952243  2769.581741 -543.458929 -2.386420e-16 -2.349716e-16   
999    999  1866.104606  2771.780972 -595.855550 -2.528761e-16 -1.911015e-16   

             VG02          VG10          VG11          VG12          VG20  \
995  8.657314e-16  9.075661e-17 -5.463122e-16 -1.148349e-15 -3.044369e-16   
996  6.613578e-16  9.518942e-18 -1.606623e-16 -5.047749e-16 -3.091435e-16   
997  4.733170e-16 -2.992207e-17 -2.724701e-16 -5.230976e-16 -2.688385e-16   
998  3.455051e-16 -6.410427e-17 -1.936275e-16 -1.818589e-16 -2.177340e-16   
999  2.267424e-16 -8.751373e-17 -1.953271e-16 -1.393487e-16 -1.448000e-16   

             VG21          VG22  
995 -1.5306

hi df_t_30
     index            x            y           z          VG00          VG01  \
995    995  1857.152681  2768.580798 -383.794333 -8.154071e-17 -2.123844e-16   
996    996  1828.739806  2774.702839 -436.489361 -8.187196e-17 -1.875308e-16   
997    997  1802.392644  2780.208676 -489.325342 -7.258194e-17 -1.738582e-16   
998    998  1780.003239  2784.669353 -542.152150 -3.304300e-17 -2.015028e-16   
999    999  1763.165977  2788.123456 -595.112634 -8.758256e-17 -2.248369e-16   

             VG02          VG10          VG11          VG12          VG20  \
995  7.957167e-16  3.513706e-16  2.066939e-16  2.287191e-16  2.089115e-16   
996  5.161698e-16  2.654756e-16  4.997418e-17  7.339724e-17  1.266573e-16   
997  3.767929e-16  2.000851e-16 -2.069951e-17 -3.123461e-17  6.842764e-17   
998  3.858251e-16  1.593503e-16 -6.451773e-17 -3.653385e-17  2.383836e-17   
999  2.545306e-16  7.945332e-17 -1.021262e-16 -2.149674e-17 -1.710215e-16   

             VG21          VG22  
995  6.9490

hi df_t_48
     index            x            y           z          VG00          VG01  \
995    995  1796.857867  2774.006809 -378.580859 -8.822939e-17 -3.002029e-17   
996    996  1753.481573  2782.531690 -431.715224 -9.112682e-18  3.632591e-18   
997    997  1712.188209  2789.985706 -484.841144 -1.172850e-17  7.178543e-18   
998    998  1676.893626  2795.724006 -538.446765 -1.402047e-17  1.773940e-17   
999    999  1650.417074  2800.155002 -592.540166 -1.493748e-17  2.492563e-17   

             VG02          VG10          VG11          VG12          VG20  \
995  1.400966e-16  6.119784e-17 -2.944245e-17 -1.586880e-16  2.622609e-16   
996  1.668040e-16  3.191480e-17 -5.560513e-17 -1.542711e-16  1.214579e-16   
997  1.305591e-16  4.681839e-17 -1.980837e-17 -1.058565e-16  8.669565e-17   
998  9.638522e-17  5.034723e-17 -2.693513e-17 -7.296974e-17  5.921449e-17   
999  4.799747e-17  4.670022e-17 -2.651947e-17 -4.641834e-17  3.551668e-17   

             VG21          VG22  
995 -2.0417

hi df_t_65
     index            x            y           z          VG00          VG01  \
995    995  1762.005558  2774.711909 -372.811553 -6.405741e-17 -1.586711e-16   
996    996  1703.084963  2785.443328 -424.669402 -2.457048e-17 -1.475495e-16   
997    997  1646.706761  2794.722146 -477.370413  7.103375e-18 -1.328457e-16   
998    998  1597.387770  2802.289923 -531.483950  3.604649e-17 -1.178150e-16   
999    999  1560.015728  2808.214440 -587.461606 -2.117026e-16 -9.481079e-17   

             VG02          VG10          VG11          VG12          VG20  \
995  2.886085e-16  3.354743e-17  6.217119e-17 -1.278181e-16 -4.683065e-16   
996  3.292852e-16  6.078939e-17  5.182550e-17 -1.034644e-16 -3.911456e-16   
997  3.596187e-16  7.635758e-17  5.156220e-17 -8.385712e-17 -3.204248e-16   
998  3.534351e-16  8.628445e-17  3.960789e-17 -8.305404e-17 -2.485296e-16   
999  2.550174e-16  5.634660e-17  2.868905e-17 -4.759749e-17 -7.030473e-17   

             VG21          VG22  
995 -1.7520

hi df_t_80
     index            x            y           z          VG00          VG01  \
995    995  1725.687245  2773.708504 -367.237508  2.508862e-17  2.367880e-17   
996    996  1653.970575  2783.713636 -417.865578  3.977822e-17  2.843312e-17   
997    997  1582.415033  2792.040633 -469.756058 -6.123877e-17  1.399683e-17   
998    998  1519.177097  2799.310516 -524.297298 -1.038787e-16  9.147290e-18   
999    999  1471.732381  2805.082449 -582.220368 -1.156953e-16  1.281536e-17   

             VG02          VG10          VG11          VG12          VG20  \
995  4.228185e-16  9.564458e-17  6.986170e-17  2.629700e-17  5.561209e-17   
996  7.079222e-16  1.181449e-16  1.043310e-16  3.681852e-17  3.315173e-17   
997  8.103212e-16  1.778757e-16  1.423739e-16  5.325237e-18 -2.816672e-17   
998  5.983264e-16  1.793700e-16  1.893487e-16 -2.626601e-17 -2.502964e-17   
999  3.519455e-16  1.828309e-16  2.189648e-16 -1.977950e-17 -2.099365e-17   

             VG21          VG22  
995 -6.1636

999  7.631123e-19 -7.131964e-18  
time: 96
hi df_t_96
     index            x            y           z          VG00          VG01  \
995    995  1699.183669  2771.914853 -357.512493 -1.806318e-17 -3.313582e-17   
996    996  1614.518368  2781.419871 -407.612445 -1.024111e-17 -2.684113e-17   
997    997  1528.223070  2790.454617 -459.333760  9.057848e-18 -1.881324e-17   
998    998  1450.991162  2798.987139 -515.236794  2.231135e-17 -5.745577e-18   
999    999  1392.206752  2805.493564 -575.657460  1.612816e-17  3.294210e-18   

             VG02          VG10          VG11          VG12          VG20  \
995  1.322651e-16 -1.782736e-17  7.111812e-17  3.135806e-18  5.798025e-17   
996  2.568752e-16 -6.245652e-18  6.128527e-17 -1.509842e-17  4.848308e-17   
997  3.890817e-16 -1.625730e-19  4.851343e-17 -5.111382e-17  1.398144e-17   
998  3.268641e-16  7.191077e-18  3.168022e-17 -5.808309e-17  4.925983e-18   
999  1.355249e-16  9.268211e-18  1.543309e-17 -2.863746e-17 -1.227567e-18   

  

In [8]:
h5_file_coord =  f'./outputs_3d_no_t_copy/lag_velocitygradient-1.h5'

with h5py.File(h5_file_coord, 'r') as file:
            # create a variable name for storing strain rate tensor array
            dataset_name = f'np_array_6'

            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                 dataset_name = file[key][:] 
                    

dataset_name[0]

HDF5-DIAG: Error detected in HDF5 (1.12.1) thread 0:
  #000: H5F.c line 620 in H5Fopen(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #001: H5VLcallback.c line 3501 in H5VL_file_open(): failed to iterate over available VOL connector plugins
    major: Virtual Object Layer
    minor: Iteration failed
  #002: H5PLpath.c line 578 in H5PL__path_table_iterate(): can't iterate over plugins in plugin path '(null)'
    major: Plugin for dynamically loaded library
    minor: Iteration failed
  #003: H5PLpath.c line 620 in H5PL__path_table_iterate_process_path(): can't open directory: /usr/local/hdf5/lib/plugin
    major: Plugin for dynamically loaded library
    minor: Can't open directory or file
  #004: H5VLcallback.c line 3351 in H5VL__file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #005: H5VLnative_file.c line 97 in H5VL__native_file_open(): unable to open file
    major: File accessibility
    minor: Unable to o

RuntimeError: Unspecified error in H5Fopen (return value <0)